# CS 470 HW3 (Asuka Li)

THIS CODE IS MY OWN WORK, IT WAS WRITTEN WITHOUT CONSULTING CODE WRITTEN BY OTHER STUDENTS. Asuka Li 

Collaboration Statement
I have consulted the following through writing this program:
- lecture slidies for CS 470
- Eligijus112's decision-tree-python project on Github

In [12]:
import pandas as pd
import numpy as np
from collections import Counter
import math
import re

class Node:
    def __init__(
        self,
        Y: list, # list of yes no's of heartdisease
        X: pd.DataFrame, # df of data, except for heartdisease col
        min_samples_split=None,
        max_depth=None,
        depth = None,
        node_type = None,
        rule = None
    ):
        self.Y = Y
        self.X = X
        
        self.min_samples_split = min_samples_split if min_samples_split else 20
        self.max_depth = max_depth if max_depth else 5
        
        self.features = list(self.X.columns) # list of attribute names
        self.depth = depth if depth else 0
        self.node_type = node_type if node_type else 'root'
        self.rule = rule if rule else ""
        self.counts = Counter(Y)
#         print(dir(self))
        self.gini_impurity = self.get_GINI()
        counts_sorted = list(sorted(self.counts.items(), key=lambda item: item[1]))
        
         # Getting the last item
        yhat = None
        if len(counts_sorted) > 0:
            yhat = counts_sorted[-1][0]
        # Saving to object attribute. This node will predict the class with the most frequent class
        self.yhat = yhat 
        
        self.n = len(Y)
        self.left = None
        self.right = None
        
        self.best_feature = None
        self.best_value = None



    def grow_tree(self):
        df = self.X.copy()
        df['Y'] = self.Y
        
#         if self.node_type == 'root':
#             node = self

        # set maxDepth as 5
        if (self.depth < 5):
            best_feature, best_value = self.best_split()
#             print(best_feature)
            
            if best_feature is not None:
#                 print(self.node_type)
                self.best_feature = best_feature
                self.best_value = best_value

                self.best_feature = best_feature
                self.best_value = best_value
                
                if df[best_feature].dtype == 'int64' or df[best_feature].dtype == 'float64':
                    left_df = df[df[best_feature] <= best_value].copy()
                    right_df = df[df[best_feature] > best_value].copy()
                    
                else:
                    left_df = df[df[best_feature] == best_value].copy()
                    right_df = df[df[best_feature] != best_value].copy()

                # delete the current best_feature from both dfs
#                 print(self.features)
#                 print(left_df)
#                 left_df.drop([best_feature], axis = 1)
#                 right_df.drop([best_feature], axis = 1)
#                 print(left_df)
                new_fea = self.features.copy()
                new_fea.remove(best_feature)
#                 print(self.features)
#                 print(new_fea)
#                 self.features.remove(best_feature)

                left = Node(
                    left_df['Y'].values.tolist(),
                    left_df[new_fea], 
                    depth=self.depth + 1, 
                    max_depth = self.max_depth,
                    min_samples_split=self.min_samples_split, 
                    node_type='left_node',
                    rule=f"{best_feature} <=/== {best_value}"
                )
                self.left = left
                self.left.grow_tree()

                right = Node(
                    right_df['Y'].values.tolist(), 
                    right_df[new_fea], 
                    depth=self.depth + 1,  
                    max_depth = self.max_depth,
                    min_samples_split=self.min_samples_split,
                    node_type='right_node',
                    rule=f"{best_feature} >/!= {best_value}"
                )
                self.right = right
                self.right.grow_tree()
                
#         return node



    def best_split(self) -> tuple:
        # calculates the best split for a decision tree
        df = self.X.copy()
        df['Y'] = self.Y

        GINI_base = self.get_GINI()

        # which split yeilds the best GINI gain?
        max_gain = 0

        best_feature = None
        best_value = None

        for feature in self.features:
            Xdf = df.sort_values(feature)
            
            # when the column is numeric variable
            if Xdf[feature].dtype == 'int64' or Xdf[feature].dtype == 'float64':
                
                # sorting the values and getting the rolling average
                xmeans = self.mavg(Xdf[feature].unique(), 2)

                for value in xmeans:
                    left_counts = Counter(Xdf[Xdf[feature] < value]['Y'])
                    right_counts = Counter(Xdf[Xdf[feature] >= value]['Y'])

                    yes_left = left_counts.get('Yes', 0)
                    no_left = left_counts.get('No', 0)
                    yes_right = right_counts.get('Yes', 0)
                    no_right = right_counts.get('No', 0)

                    gini_left = self.GINI_impurity(yes_left, no_left)
                    gini_right = self.GINI_impurity(yes_right, no_right)

                    n_left = yes_left + no_left
                    n_right = yes_right + no_right
                    w_left = n_left / (n_left + n_right)
                    w_right = n_right / (n_left + n_right)

                    # weighted GINI impurity
                    wGINI = w_left*gini_left + w_right*gini_right

                    GINIgain = GINI_base - wGINI

                    if GINIgain > max_gain:
                        best_feature = feature
                        best_value = value
                        max_gain = GINIgain
            
            # when the columns are categorical
            else :
                cat_list = Xdf[feature].unique().tolist()
                for cat in cat_list:
                    left_counts = Counter(Xdf[Xdf[feature] == cat]['Y'])
                    right_counts = Counter(Xdf[Xdf[feature] != cat]['Y'])
                    
                    yes_left = left_counts.get('Yes', 0)
                    no_left = left_counts.get('No', 0)
                    yes_right = right_counts.get('Yes', 0)
                    no_right = right_counts.get('No', 0)

                    gini_left = self.GINI_impurity(yes_left, no_left)
                    gini_right = self.GINI_impurity(yes_right, no_right)

                    n_left = yes_left + no_left
                    n_right = yes_right + no_right
                    w_left = n_left / (n_left + n_right)
                    w_right = n_right / (n_left + n_right)

                    # weighted GINI impurity
                    wGINI = w_left*gini_left + w_right*gini_right

                    GINIgain = GINI_base - wGINI

                    if GINIgain > max_gain:
                        best_feature = feature
                        best_value = cat
                        max_gain = GINIgain
                    

        return (best_feature, best_value)  


    def get_GINI(self):
        yes_count = self.counts.get('Yes', 0)
        no_count = self.counts.get('No', 0)
        
#         print(yes_count)
#         print(no_count)
        return self.GINI_impurity(yes_count, no_count)

    
    @staticmethod
    def GINI_impurity(yes_count: int, no_count: int) -> float:
        
        n = yes_count + no_count

        if n == 0:
            return 0.0

        p1 = yes_count / n
        p2 = no_count / n

        gini = 1 - (p1**2 + p2**2)

        return gini

    
    @staticmethod
    def mavg(x: np.array, window: int) -> np.array:
        # calculates the moving average of the given list
        return np.convolve(x, np.ones(window), 'valid')
 

    def predict(self, X:pd.DataFrame):
        predictions = []

        for _, x in X.iterrows():
            values = {}
            for feature in self.features:
                values.update({feature: x[feature]})
        
            predictions.append(self.predict_obs(values))
        return predictions
    
    
    def predict_obs(self, values: dict) -> int:
        # predict a class given a set of features
        cur_node = self
        
        while cur_node.depth < cur_node.max_depth:
            best_feature = cur_node.best_feature
            best_value = cur_node.best_value
            
            if cur_node.n < cur_node.min_samples_split:
                break
            
#             print(best_feature)
#             print(type(best_feature) == str)
#             print(best_value)
#             print("best_value type: ", type(best_value))
#             print(isinstance(best_value, np.float))
#             print(values.get(best_feature))

#            or type(best_value) == str
            if type(best_feature) != str and not (isinstance(best_value, np.float)):
                break
            
            if isinstance(best_value, np.float):
                if (values.get(best_feature) <= best_value):
#                     print("entered")
                    if self.left is not None:
                        cur_node = cur_node.left
                else:
                    if self.right is not None:
                        cur_node = cur_node.right
                    
            elif type(best_value) == str:
                if (values.get(best_feature) == best_value):
                    if self.left is not None:
                        cur_node = cur_node.left
                else:
                    if self.right is not None:
                        cur_node = cur_node.right
            
#             v = values.get(best_feature)
# #             print(v)
#             if v is not None: 
#                 if (type(v) == 'int' or type(v) == 'float') & (v < best_value):
#                     if self.left is not None:
#                         cur_node = cur_node.left
#                 elif (type(v) != 'int' or type(v) != 'float') & (v == best_value):
#                     if self.left is not None:
#                         cur_node = cur_node.left
#                 else:
#                     if self.right is not None:
#                         cur_node = cur_node.right
#         print(cur_node.yhat)            
        return cur_node.yhat

In [13]:
def genDecisionTree(data, param2, param3, out):
    # extracting what's in the training set
    file1 = open(param2, 'r')
    lines = file1.readlines()
    
    nlist = []
    for l in lines:
        nlist.append(l.strip('\n'))
    
    nlist = list(map(int, nlist))
#     print(nlist)
    trainset = data[data['personID'].isin(nlist)]
    

    Y = data['heartDisease'].values
    X = data[['age', 'gender', 'chestPainType', 'restingBloodPressure', 
               'serumCholesterol', 'fastingBloodSugar', 'restingElectrocardiographicResults', 'maxHeartRateAchieved', 'exerciseInducedAngina',
              'oldpeak', 'slope', '#majorVesselsColored']]

    features = list(X.columns)

    hp = {
     'max_depth': 7,
     'min_samples_split': 50
    }
    root = Node(Y, X, **hp)

    root.grow_tree()

    
    # extract what's in the testing set
    file1 = open(param3, 'r')
    lines = file1.readlines()
    
    nlist = []
    for l in lines:
        nlist.append(l.strip('\n'))
    
    nlist = list(map(int, nlist))
#     print(nlist)
    testset = data[data['personID'].isin(nlist)]
    
    # predicting
    Xsubset = testset[['personID', 'age', 'gender', 'chestPainType', 'restingBloodPressure', 
               'serumCholesterol', 'fastingBloodSugar', 'restingElectrocardiographicResults', 'maxHeartRateAchieved', 'exerciseInducedAngina',
              'oldpeak', 'slope', '#majorVesselsColored']]
    Xsubset['yhat'] = root.predict(Xsubset)
    print(Xsubset['yhat'])
    
    # writing to output file
    with open(out, 'w') as f:
        for n in nlist:
            print(n)
            f.write(str(n))
            f.write(" ") 
            d = Xsubset[Xsubset['personID']==n]['yhat']
#             print(d.to_string())
            d = re.sub(r'\d+', '', d.to_string())
            d = re.sub(' ', '', d)
            f.write(d)
            f.write('\n')

In [14]:
data = pd.read_csv("data.csv")

data.columns =['personID', 'age', 'gender', 'chestPainType', 'restingBloodPressure', 
           'serumCholesterol', 'fastingBloodSugar', 'restingElectrocardiographicResults', 'maxHeartRateAchieved', 'exerciseInducedAngina',
          'oldpeak', 'slope', '#majorVesselsColored', 'heartDisease']

# print("about to generate decision tree")
genDecisionTree(data, "para2_file.txt", "para3_file.txt", "para4_file.txt")


1       No
3      Yes
4       No
5      Yes
6      Yes
      ... 
294     No
295     No
296     No
299    Yes
301     No
Name: yhat, Length: 157, dtype: object
0
2
4
5
6
7
8
10
12
13
15
18
19
20
21
24
26
27
29
30
31
32
33
35
36
37
38
41
42
44
45
47
48
49
54
55
56
57
65
66
67
69
71
73
74
76
78
80
83
84
88
89
90
93
94
95
96
97
102
105
112
116
117
122
123
124
126
127
129
130
132
134
136
137
142
143
145
148
149
156
157
159
161
165
166
170
171
174
176
177
178
179
182
184
186
188
191
193
194
196
197
199
201
202
204
205
206
207
208
212
213
214
215
216
219
220
221
222
223
224
227
232
233
238
239
242
243
244
245
248
251
253
254
255
258
259
262
263
264
267
269
271
272
274
275
278
279
282
286
288
290
292
294
295
296
297
300
302
